In [ ]:
import requests
import os
import pandas as pd

gathering the first dataset

#make a file path for the file if it is not available 
folder_name = 'image_predictions_folder'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

In [ ]:
#opening the file 
with open(os.path.join(folder_name,
                      url.split('/')[-1]) , mode='wb') as file:
    file.write(response.content)

gathering the second dataset

In [ ]:
twit_arc = pd.read_csv('twitter-archive-enhanced.csv')
twit_arc.info()

gathering the third dataset

THE cell below is used to query twitter api to get the favorite_count and 
retweet_count for the using the tweet id from the second dataset.
NB. the scret tokens are incomplete in compliance with the terms and conditions of twitter.

In [ ]:


import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are are not complete to comply with Twitter's API terms and conditions
access_token= '153954C'
access_secret = 'R1hOD'
consumer_key  = '7xdHjD'
consumer_secret= 'CWz8uu\'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

tweet_ids = df_1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except  tweepy.errors.TweepyException as e: 
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [ ]:
import pandas as pd
image_pred = pd.read_csv('image_predictions_folder/image-predictions.tsv', sep='\t')
image_pred.head()


In [ ]:
twit_arc = pd.read_csv('twitter-archive-enhanced.csv')
twit_arc.info()

In [ ]:
api_tweet = pd.read_json("tweet-json.txt", lines = True,dtype ={"id_str": str})[:-1]
api_tweet

After viewing the json data in a text editor acquired using twitter api i have
decided to extract only the favorite count and retweet count.


In [ ]:
import json
df_list = []


with open('tweet-json.txt', 'r') as file:
    for line in file:
        tweets = json.loads(line)
        ids =  {'tweet_id':tweets['id'] ,
                'favorite_count': tweets['favorite_count'], 
                'retweet_count':tweets['retweet_count']}
        df_list.append(ids)
tweet_api = pd.DataFrame(df_list)
tweet_api


we now have three data frames which are
 1. image_pred
 2. twit_arc
 3. tweet_api

# ASSESSING

###### Assesing the image prediction file

In [ ]:
#visual assesment
image_pred.sample(10)

In [ ]:
image_pred.head()

issues

* inconsistent case-type for values p1,p2 and p3 columns  

In [ ]:
image_pred.info()

issues

* tweet_id should be string not int
* jpg_url should be image_url so that it can be descriptive

In [ ]:
image_pred.sample(15)

issues

* some rows have p1_dog,p2_dog,p3_dog ratings as False

##### Assesing the tweet from twitter api

In [ ]:
tweet_api.head()

In [ ]:
tweet_api.info()

issues

* tweet_id should be string not int
* favorite count and retweet count should be int not float

Assessing the twitter archive enhanced dataset

In [ ]:
twit_arc

In [ ]:
twit_arc.info()

Issues

* tweet_id should str not int
* timestamp should be "data time" not "str"
* we have 2356 entries here which means the data in the other two dataset are incomplete.

In [ ]:
twit_arc.query('rating_numerator > 15')['rating_numerator']

In [ ]:
twit_arc['rating_numerator'].describe()

In [ ]:
twit_arc['rating_numerator'].value_counts()

In [ ]:
# visual assesment
twit_arc.sample(15)

In [ ]:
twit_arc.query('rating_numerator >15')[['rating_numerator','text']]

In [ ]:
twit_arc.query('rating_denominator != 10 ')[['rating_denominator','text']]

issues
* 466 numerator ratings are less than 10 0r greater than 15.Since 15 is the highest rating on weratedog on twitter.
* 23 denominator rating are not equal to 10 
* Nulls represented as (none) in name column 
* some columns are not needed in this dataset.

issues 
* doggo, flo0fer,pupper,puppo are dog stages which should be under a variable.

#### Grouping the issues

QUALITY ISSUES
* image_pred
    - inconsistent case-type for values p1,p2 and p3 columns. 
    - tweet_id should be string not int.
    - incomplete data.
###### 
* twit_arc
    - tweet_id should be string not int.
    - incomplete data.   
######  
* tweet_api dataset
    - some numerator ratings are greater than 15 from vissual assesment.Since 15 is the highest rating on weratedog twitter.
    - 23 denominator rating are not equal to 10.
    - Nulls represented as (none) in some columns. 
    - some columns are not needed in this dataset for our insight and visualization.
    - tweet_id is duplicated
    
    
TIDINESS ISSUES
* image_pred
    - jpg_url should be image_url so that it can be descriptive (tidy)
    - p_conf is not descriptive of the columns in the image_pred dataset.   
###### 
* tweet_api dataset
    - doggo, floofer,pupper,puppo are dog stages which should be under a variable.
    - all dataset shoud be 1 dataset.


# CLEANING

In [ ]:
# make copies of the dataset so as to preserve the original dataset.
twit_arc_cln = twit_arc.copy()
img_pred_cln = image_pred.copy()
tweet_api_cln = tweet_api.copy()

## Tidiness
#### twit_arc:doggo, floofer,pupperand puppo are dog stages which should be under a variable.
 
#### Define
- extract the stage of dogs from the tweet text irrespective of the case_type.
- drop the doggo, floofer,pupper and puppo 
- convert the values in the stage of dogs to lower-case for consistency.

#### Code

In [ ]:
import re
twit_arc_cln['dog_stage'] = twit_arc_cln['text'].str.extract(r'(doggo|floofer|pupper|puppo)', flags = re.IGNORECASE)
twit_arc_cln = twit_arc_cln.drop(columns=['doggo','floofer','pupper','puppo'])
twit_arc_cln['dog_stage'] =twit_arc_cln['dog_stage'].str.lower()

#### Test

In [ ]:
twit_arc_cln.sample(12)

In [ ]:
twit_arc_cln['dog_stage'].value_counts()

## Tidiness
#### image_pred_cln and tweet_api_cln:
- all dataset shoud be 1 dataset 
- tweet_id is duplicated 
#### Define
* merge the three dataset together on the tweet_id column.

#### Code

In [ ]:
twit_arc_cln = pd.merge(twit_arc_cln, img_pred_cln , on= "tweet_id" , how="left") 
new_twit_arc_cln = pd.merge(twit_arc_cln , tweet_api_cln , on= "tweet_id" , how="left")

#### Test

In [ ]:
new_twit_arc_cln

In [ ]:
merged_twit = new_twit_arc_cln.copy()
merged_twit.info()

## Tidiness 
#### some columns are not needed for the insights we want to make.
#### Define
* drop unused columns

#### Code

In [ ]:
merged_twit.columns

In [ ]:
merged_twit.drop(columns=['in_reply_to_status_id','in_reply_to_user_id',
                          'retweeted_status_id', 'retweeted_status_user_id',
                          'retweeted_status_timestamp', 'expanded_urls'], inplace=True)

#### Test

In [ ]:
merged_twit.sample(10)

## Quality 
#### null values represented as none
#### Define
* replace the none values as np.nan

#### Code


In [ ]:
import numpy as np
merged_twit = merged_twit.replace("None", value = np.nan )


#### Test

In [ ]:
merged_twit

## Quality 
#### 23 denominator rating are not equal to 10
#### Define
* change all the values in the denominator rating to 10 since the ratings always 10 .

#### Code


In [ ]:
merged_twit.rating_denominator = 10

#### Test

In [ ]:
merged_twit.rating_denominator.value_counts()

## Quality 
#### some numerator ratings greater than 15
#### Define
* check for how many numerator ratings greater than 15.
* change it manually if possible otherwise programatically.

#### Code

In [ ]:

less_15 = merged_twit.query('rating_numerator > 15')['rating_numerator']
#to get the index of the values
val = less_15.index.values.tolist()
len(val)

In [ ]:
merged_twit.query('rating_numerator > 15')[['text']]


In [ ]:
# from the above cells there are 26 cells with numerator ratings greater than 15 
# we go through the text and get the rating_numerator from it 
# some of the ratings are group ratings so we find the ratio. the others are 
#replaced by the median of the numerator ratings which is 11


#list of correct ratings from the text
correct_list = [13,11,11,11,13,9.75,12,11,9.75,11.27,11,11,12,11,11,10,9,12,11,11,12.5,11.26,12,11,11,12]

for i in range(26):
        merged_twit.rating_numerator[val[i]]=correct_list[i]
    

In [ ]:
merged_twit.rating_numerator.median()

#### Test

In [ ]:
merged_twit.rating_numerator.value_counts()


In [ ]:
merged_twit.head()

## Quality 
####  inconsistent case-type for values p1,p2 and p3 columns
#### Define
* change the case of all the data in p1,p2,p3 to title case

#### Code

In [ ]:
merged_twit.p1 = merged_twit.p1.str.title()
merged_twit.p2 = merged_twit.p2.str.title()
merged_twit.p3 = merged_twit.p3.str.title()

#### Test

     

In [ ]:
merged_twit[['p1','p2','p3']].sample(5)

## Quality
##### tweet_id should string not int
#####  timestamp should be date_time not str
#### favorite count and retweet count should be int not float
#### Define
* change the type of tweet_id to str
* convert the column type using to_datetime method 
* convert favorite count and retweet count type to int 
#### Code

In [ ]:
merged_twit['tweet_id'] = merged_twit['tweet_id'].astype(str)
merged_twit['timestamp']= pd.to_datetime(merged_twit['timestamp'])


#### Test

In [ ]:
merged_twit.info()

## Quality
##### jpg_url should be image_url so that it can be descriptive (tidy)
#####    p1_conf,p2_conf,p3_conf is not descriptive of the columns in the  dataset.

#### Define
* rename the columns

#### Code

In [ ]:
merged_twit.rename(columns={'p1_conf': 'p1_confidence'}, inplace=True)
merged_twit.rename(columns={'p2_conf': 'p2_confidence'}, inplace=True)
merged_twit.rename(columns={'p3_conf': 'p3_confidence'}, inplace=True)
merged_twit.rename(columns={'jpg_url': 'image_url'}, inplace=True)

#### Test

In [ ]:
merged_twit.head()

#### save data to master_twitter_archive 

In [ ]:
merged_twit.to_csv('master_twitter_archive.csv', index=False)

# Analysis And visulization.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df= pd.read_csv('master_twitter_archive.csv')

In [ ]:
df.info()

###  The top 10 most predicted breed of dog from  all the three predictions

In [ ]:
# get the value_counts for all breed from three predictions
dfc = df.p1.value_counts()+df.p2.value_counts()+df.p3.value_counts()
dfc.sort_values(ascending = False)[:10].plot(kind = 'bar',width = 0.8, figsize = (10,5));
plt.xlabel('Dog Breed',fontsize= 14),plt.ylabel('Number of Pictures',fontsize= 14),
plt.title('Top 10 breeds with the most number of pictures'.title(),fontsize= 14);



### percentage of pictures predicted to be dog's by each prediction .


In [ ]:
def is_dog(pred):
    return df[pred][df[pred] == True].count()/df[pred].count()*100

In [ ]:
print(f'percentage of pictures predicted to be dog pictures by p1 is {is_dog("p1_dog")}')
print(f'percentage of pictures predicted to be dog pictures by p2 is {is_dog("p2_dog")}')
print(f'percentage of pictures predicted to be dog pictures by p3 is {is_dog("p3_dog")}')

In [ ]:
plt.pie([is_dog('p1_dog'),is_dog('p2_dog'),is_dog('p3_dog')],labels=('p1','p2','p3'), explode = [0.1,0.1,0.1],autopct ='%1.1f%%')
plt.title("percentage of pictures predicted to be dog's by each prediction".title(), fontsize= 14)
plt.show()

##### Insight
From this we can seee that the probility of a picture being predicted to be dog picture for the three predictions are closed.

### Most common rating 

In [ ]:
df.rating_numerator.value_counts().plot(kind= 'bar', figsize = (10,6));
plt.xlabel('Ratings',fontsize= 14,);
plt.ylabel('Pictures',fontsize= 14);
plt.title('Number of pictures for each rating',fontsize= 14);

##### Insight
Rating of 12 is the most common rating given to dogs 

### Dog Stage with the most number of pictures

In [ ]:
df.dog_stage.value_counts().plot(kind= 'bar');


dog that are at the stage of pupper are the most followed by doggo, puppo and floofer.

In [ ]:
def dog_stage(col):
    g = df.groupby('dog_stage')[col].mean().sort_values()
    g.plot(kind = 'bar')
    plt.ylabel(col.title(),fontsize= 14,);
    plt.xlabel('Dog_stage',fontsize= 14);
    plt.title(f'Average {col} Per Dog_Stage',fontsize= 14);

    


### Dog Stage with the highest average rating

In [ ]:
dog_stage('rating_numerator')

On average picture in 'puppo' dog stage get most rating. 

### Dog Stage with the highest average favorite_count

In [ ]:

dog_stage('favorite_count')

##### Insight
On average picture in 'puppo' dog stage get most likes.

### Relationship between favorite_count , retweet_count and rating

In [ ]:
def plot_rel(col_a,col_b) : 
    """
    Parameters
    ----------
    col_a: first column
    col_b : second column.

    Returns
    ----------
    A scatter chart showing the correlation between col_a and col_b

    """
        
    #import statsmodels.api as sn
    import scipy  
    #To get the statistacal correlation between the two columns, we need to drop the null valuues in the two columns
    c = df[[col_a,col_b]].dropna()
    print(f'correlation between {col_a} and {col_b} is' , scipy.stats.pearsonr(c[col_a],c[col_b])[0])
    sns.set_theme(color_codes=True)
    #To plot a scatter chart
    sns.regplot(x=col_a, y= col_b, data=c, marker="o", 
                scatter_kws={"s": 50 });
    sns.set(rc={'figure.figsize':(6,4)})
    plt.xlabel(col_a.title(), fontsize=12)
    plt.ylabel(col_b.title(), fontsize=12);
    plt.title(f'relationship between {col_a} and {col_b}'.title(), fontsize=12);
    
    #print('correlation between col_a and col_b is' 

In [ ]:
print('PLOT 1')
# Relationship between favorite_count and retweet_count
plot_rel('retweet_count','favorite_count')

In [ ]:
print('PLOT 2')
# Relationship between favorite_count and retweet_count
plot_rel('rating_numerator','favorite_count')

### Summary
* from plot 1. there is ahigh correlation of 0.703 between favorite count and retweet count.  
* From the two plots below puppo has most likes and ratings which means pictures in puppo stage tend to get more likes and ratings and equally more retweets on average since their is a high correlation  between favorite_count and retweet_count

### Limitations.

* p1, p2,p3 are 100% accurate which means some of their prediction might be r=wrong which would have affected our result.
* Also many ofthe pictures dont have dog_stages which would have affected the result of our analysis.